# 🌍 Overview

This demo is a minimalistic MLOps project intended to showcase how to put ML workflows in production. It features: 

- A feature engineering pipeline that loads data and prepares it for training.
- A training pipeline that loads the preprocessed dataset and trains a model.
- A batch inference pipeline that runs predictions on the trained model with new data.
- A stack switching and leveraging of Sagemaker step operator to outsource training to Cloud
- An analysis of training artifacts and their lineage (including connection with W&B)

<img src="_assets/pipeline_overview.png" width="50%" alt="Pipelines Overview">

# 👶 Step 0. Install Requirements

Let's install ZenML to get started. First we'll install the latest version of
ZenML as well as the `sklearn` and `xgboost` integration of ZenML:

In [ ]:
! pip3 install -r requirements.txt
! zenml integration install sklearn xgboost -y
! zenml connect --url https://1cf18d95-zenml.cloudinfra.zenml.io 

import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
# Initialize ZenML and set the default stack
!zenml init
!zenml stack set local-sagemaker-step-operator-wandb

In [ ]:
# Do the imports at the top
from zenml import Model
from zenml.client import Client
from zenml.logger import get_logger

from pipelines import training, inference

logger = get_logger(__name__)

# Initialize the ZenML client to fetch objects from the ZenML Server
client = Client()

# ⌚ Step 1: Training pipeline

Now that we have our data it makes sense to train some models to get a sense of
how difficult the task is. The Breast Cancer dataset is sufficiently large and complex 
that it's unlikely we'll be able to train a model that behaves perfectly since the problem 
is inherently complex, but we can get a sense of what a reasonable baseline looks like.

We'll start with two simple models, a SGD Classifier and a Random Forest
Classifier, both batteries-included from `sklearn`. We'll train them both on the
same data and then compare their performance.

<img src="_assets/training_pipeline.png" width="50%" alt="Training pipeline">

In [ ]:
# let's have a look at model training step
%pycat steps/model_trainer.py

Our two training steps both return different kinds of classifier
models, so we use the generic `ClassifierMixin` type hint for the return type.

ZenML allows you to load any version of any dataset that is tracked by the framework
directly into a pipeline using the `Client().get_artifact_version` interface. This is very convenient
in this case, as we'd like to send our preprocessed dataset from the older pipeline directly
into the training pipeline.

In [ ]:
# let's have a look at training pipeline
%pycat pipelines/training.py

The end goal of this quick baseline evaluation is to understand which of the two
models performs better. We'll use the `evaluator` step to compare the two
models. This step takes in the model from the trainer step, and computes its score
over the testing set.

Soon you will see that it is relatively easy to train ML models using ZenML pipelines. But it can be somewhat clunky to track
all the models produced as you develop your experiments and use-cases. Luckily, ZenML offers a *Model Control Plane*,
which is a central register of all your ML models.

You can easily create a ZenML Model and associate it with your pipelines using the `Model` object:

In [ ]:
pipeline_settings = {}

# Lets add some metadata to the model to make it identifiable
pipeline_settings["model"] = Model(
    name="breast_cancer_classifier",
    license="Apache 2.0",
    description="A breast cancer classifier",
)

In [ ]:
# Let's train the XGBoost model and tag the version name with "xgboost"
pipeline_settings["model"].tags = ["breast_cancer", "classifier", "xgboost"]

# Use an XGBoost model with fixed seed.
training.with_options(enable_cache=False,**pipeline_settings)(
    model_type="xgboost",
    random_state=42
)

xgboost_run = client.get_pipeline("training").last_run

In [ ]:
# Let's train the SGD model and tag the version name with "sgd"
pipeline_settings["model"].tags = ["breast_cancer", "classifier", "sgd"]

# Use a SGD classifier
sgd_run = training.with_options(enable_cache=False,**pipeline_settings)(
    model_type="sgd",
    random_state=42
)

sgd_run = client.get_pipeline("training").last_run

You can see from the logs already how our model training went: the
`XGBClassifier` performed considerably better than the `SGDClassifier`.
We can use the ZenML `Client` to verify this:

In [ ]:
# The evaluator returns a float value with the accuracy
xgboost_run.steps["model_evaluator"].output.load() >= sgd_run.steps["model_evaluator"].output.load()

Running both pipelines has created two associated **model versions**.
You can list your ZenML model and their versions as follows:

In [ ]:
zenml_model = client.get_model("breast_cancer_classifier")
print(zenml_model)

versions = zenml_model.versions

print(f"Model {zenml_model.name} has {len(versions)} versions")

versions[-2].version, versions[-1].version

The interesting part is that ZenML went ahead and linked all artifacts produced by the
pipelines to that model version, including the two pickle files that represent our
SGD and RandomForest classifier. We can see all artifacts directly from the model
version object:

In [ ]:
# Let's load the XGBoost version
xgboost_zenml_model_version = client.list_model_versions("breast_cancer_classifier", tag="xgboost")[-1]

# We can now load our classifier directly as well
xgboost_classifier = xgboost_zenml_model_version.get_artifact("breast_cancer_classifier").load()

xgboost_classifier

If you are a [ZenML Cloud](https://zenml.io/cloud) user, you can see all of this visualized in the dashboard:

<img src="_assets/cloud_mcp_screenshot.png" width="70%" alt="Model Control Plane">

There is a lot more you can do with ZenML models, including the ability to
track metrics by adding metadata to it, or having them persist in a model
registry. However, these topics can be explored more in the
[ZenML docs](https://docs.zenml.io).

For now, we will use the ZenML model control plane to promote our best
model to `production`. You can do this by simply setting the `stage` of
your chosen model version to the `production` tag.

In [ ]:
# Set our best classifier to production
xgboost_zenml_model_version.set_stage("production", force=True)

Of course, normally one would only promote the model by comparing to all other model
versions and doing some other tests. But that's a bit more advanced use-case. See the
[e2e_batch example](https://github.com/zenml-io/zenml/tree/main/examples/e2e) to get
more insight into that sort of flow!

<img src="_assets/cloud_mcp.png" width="60%" alt="Model Control Plane">

Once the model is promoted, we can now consume the right model version in our
batch inference pipeline directly. Let's see how that works.

# 🫅 Step 2: Consuming the model in production

The batch inference pipeline simply takes the model marked as `production` and runs inference on it
with `live data`. The critical step here is the `inference_predict` step, where we load the model in memory
and generate predictions:

<img src="_assets/inference_pipeline.png" width="45%" alt="Inference pipeline">

In [ ]:
# let's have a look at training pipeline
%pycat steps/inference_predict.py


Apart from the loading the model, we must also load the preprocessing pipeline that we ran in feature engineering,
so that we can do the exact steps that we did on training time, in inference time. Let's bring it all together:

In [ ]:
# let's have a look at training pipeline
%pycat pipelines/inference.py

The way to load the right model is to pass in the `production` stage into the `Model` config this time.
This will ensure to always load the production model, decoupled from all other pipelines:

In [ ]:
pipeline_settings = {"enable_cache": False}

# Lets add some metadata to the model to make it identifiable
pipeline_settings["model"] = Model(
    name="breast_cancer_classifier",
    version="production", # We can pass in the stage name here!
)

In [ ]:
# the `with_options` method allows us to pass in pipeline settings
#  and returns a configured pipeline
inference.with_options(**pipeline_settings)()

ZenML automatically links all artifacts to the `production` model version as well, including the predictions
that were returned in the pipeline. This completes the MLOps loop of training to inference:

In [ ]:
# Fetch production model
production_model_version = client.get_model_version("breast_cancer_classifier", "production")

# Get the predictions artifact
production_model_version.get_artifact("predictions").load()

You can also see all predictions ever created as a complete history in the dashboard:

<img src="_assets/cloud_mcp_predictions.png" width="70%" alt="Model Control Plane">

# 🐙 Step 3: Analyzing results

In [ ]:
sgd_model_version = client.list_model_versions("breast_cancer_classifier",tag="sgd")[-1]
xgboost_model_version = client.list_model_versions("breast_cancer_classifier",tag="xgboost")[-1]
print(f"SGD version is staged as `{sgd_model_version.stage}`")
print(f"XGBoost version is staged as `{xgboost_model_version.stage}`")

At first, let's pull some meta information collected during models evaluation stage. To recall we used this step as evaluator:
```python
@step
def model_evaluator(
    model: ClassifierMixin,
    dataset_trn: pd.DataFrame,
    dataset_tst: pd.DataFrame,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
    target: Optional[str] = "target",
) -> float:
    # Calculate the model accuracy on the train and test set
    trn_acc = model.score(...)
    tst_acc = model.score(...)

    ...
    
    predictions = model.predict(dataset_tst.drop(columns=[target]))
    metadata = {
        "train_accuracy": float(trn_acc),
        "test_accuracy": float(tst_acc),
        "confusion_matrix": confusion_matrix(dataset_tst[target], predictions)
        .ravel()
        .tolist(),
    }
    log_model_metadata(metadata={"wandb_url": wandb.run.url})
    log_artifact_metadata(
        metadata=metadata,
        artifact_name="breast_cancer_classifier",
    )

    wandb.log({"train_accuracy": metadata["train_accuracy"]})
    wandb.log({"test_accuracy": metadata["test_accuracy"]})
    wandb.log(
        {
            "confusion_matrix": wandb.sklearn.plot_confusion_matrix(
                dataset_tst[target], predictions, ["No Cancer", "Cancer"]
            )
        }
    )
    return float(tst_acc)
```
First we pull Accuracy metrics out of both model version for comparison:

In [ ]:
sgd_clf_metadata = sgd_model_version.get_artifact("breast_cancer_classifier").run_metadata
xgboost_clf_metadata = xgboost_model_version.get_artifact("breast_cancer_classifier").run_metadata
print(f"SGD{' (production)' if sgd_model_version.stage == 'production' else ''} metrics: train={sgd_clf_metadata['train_accuracy'].value*100:.2f}% test={sgd_clf_metadata['test_accuracy'].value*100:.2f}%")
print(f"XGBoost{' (production)' if xgboost_model_version.stage == 'production' else ''} metrics: train={xgboost_clf_metadata['train_accuracy'].value*100:.2f}% test={xgboost_clf_metadata['test_accuracy'].value*100:.2f}%")

Now lets' plot collected Confusion Matrixes:

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(metadata_pointer, tp: str,ax):
    confusion_matrix = np.array(metadata_pointer["confusion_matrix"].value, dtype=float).reshape((2,2))
    confusion_matrix /= np.sum(confusion_matrix)
    sns.heatmap(confusion_matrix, annot=True,fmt='.2%',cmap="coolwarm",ax=ax)
    ax.set_title(f"{tp} confusion matrix")
    ax.set_ylabel("Ground Label")
    ax.set_xlabel("Predicted Label")

fig, ax = plt.subplots(1,2,figsize=(15,4))
plot_confusion_matrix(sgd_clf_metadata, "SGD",ax[0])
plot_confusion_matrix(xgboost_clf_metadata, "RF",ax[1])

So far we were able to collect all the information we tracked using Model Control Plane, but we also had Weights&Biases tracking enabled - let's dive into.

Thanks to Model Control Plane metadata we establish a nice connection between those 2 entities:

In [ ]:
print(f'SGD version: {sgd_model_version.run_metadata["wandb_url"].value}')
print(f'XGBoost version: {xgboost_model_version.run_metadata["wandb_url"].value}')

With Model Control Plane we can also easily track lineage of artifacts and pipeline runs:

In [ ]:
for artifact_name, versions in sgd_model_version.data_artifacts.items():
    if versions:
        print(f"Existing version of `{artifact_name}`:")
        for version_name, artifact_ in  versions.items():
            print(version_name, artifact_.data_type.attribute)

In [ ]:
for run_name, run_ in sgd_model_version.pipeline_runs.items():
    print(run_name, run_.id)

# 🙏 Step 4: Moving to production

Let's move some heavy lifting to the Sagemaker. This can be achieved using Sagemaker orchestrator.

<img src="_assets/sagemaker_stack.png" width="60%" alt="Sagemaker stack">

In [ ]:
!zenml stack set sagemaker-pipelines-wandb
!zenml stack describe sagemaker-pipelines-wandb

In [ ]:
client.activate_stack("sagemaker-pipelines-wandb")

pipeline_settings = {}

pipeline_settings["model"] = Model(
    name="breast_cancer_classifier",
    license="Apache 2.0",
    description="A breast cancer classifier",
    tags = ["breast_cancer", "classifier", "xgboost"]
)

training.with_options(**pipeline_settings)(
    model_type="xgboost",
    random_state=42
)

# Step 100: deploy

In [109]:
import os
from typing import Tuple, Any
from typing_extensions import Annotated

import boto3
import sagemaker

from zenml import pipeline, step, get_pipeline_context, get_step_context
from sagemaker.image_uris import retrieve
from datetime import datetime
from sagemaker.predictor import Predictor
import pandas as pd
from steps import data_loader, inference_preprocessor
import random

client.activate_stack("local-sagemaker-step-operator-wandb")


def get_aws_config() -> Tuple[Any, sagemaker.Session, str]:
    REGION_NAME = os.getenv("AWS_REGION", "us-east-1")
    ROLE_NAME = os.getenv("AWS_ROLE_NAME", "hamza_connector")
    os.environ["AWS_DEFAULT_REGION"] = REGION_NAME

    auth_arguments = {
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID", None),
        "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY", None),
        "aws_session_token": os.getenv("AWS_SESSION_TOKEN", None),
        "region_name": REGION_NAME,
    }
    iam = boto3.client("iam", **auth_arguments)
    role = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]
    session = sagemaker.Session(boto3.Session(**auth_arguments))

    return role, session, REGION_NAME


@step(enable_cache=False)
def deploy_endpoint() -> Annotated[str, "sagemaker_endpoint_name"]:
    role, session, region = get_aws_config()

    model = get_step_context().model._get_model_version()
    if "sgd" in {t.name for t in model.tags}:
        image_uri = retrieve(
            region=region, framework="sklearn", version="1.0-1"
        )
        entry_point = "utils/sklearn_inference.py"
    else:
        image_uri = retrieve(
            region=region, framework="xgboost", version="1.5-1"
        )
        entry_point = None

    model_data = (
        f'{model.get_artifact("breast_cancer_classifier").uri}/model.tar.gz'
    )

    endpoint_name = f'breast-cancer-classifier-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")}'
    sagemaker.Model(
        image_uri=image_uri,
        model_data=model_data,
        sagemaker_session=session,
        role=role,
        entry_point=entry_point,
    ).deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name=endpoint_name,
    )
    return endpoint_name


@step
def predict_on_endpoint(
    endpoint_name: str, dataset: pd.DataFrame
) -> Annotated[pd.Series, "real_time_predictions"]:
    predictor = Predictor(endpoint_name=endpoint_name)
    predictions = predictor.predict(
        data=dataset.to_csv(header=False, index=False),
        initial_args={"ContentType": "text/csv"},
    )
    return pd.Series(
        [float(l) for l in predictions.decode().split("\n") if l],
        name="predictions",
    )


@step
def shutdown_endpoint(endpoint_name: str):
    predictor = Predictor(endpoint_name=endpoint_name)
    predictor.delete_endpoint()


@pipeline
def deploy(shutdown_endpoint_after_predicting: bool = True):
    # Get the preprocess pipeline artifact associated with this version
    preprocess_pipeline = get_pipeline_context().model.get_artifact(
        "preprocess_pipeline"
    )

    df_inference = data_loader(
        random_state=random.randint(0, 1000), is_inference=True
    )
    df_inference = inference_preprocessor(
        dataset_inf=df_inference,
        preprocess_pipeline=preprocess_pipeline,
        target="target",
    )
    endpoint_name = deploy_endpoint()
    predict_on_endpoint(endpoint_name, df_inference)
    if shutdown_endpoint_after_predicting:
        shutdown_endpoint(endpoint_name, after=["predict_on_endpoint"])


# deploy.with_options(
#     model=Model(name="breast_cancer_classifier", version="latest")
# )()

In [112]:
deploy.with_options(
    model=Model(name="breast_cancer_classifier", version="19")
)()

version 19 is numeric and will be fetched using version number.
version 19 is numeric and will be fetched using version number.
Initiating a new run for the pipeline: deploy.


Reusing registered pipeline version: (version: 5).
Executing a new run.
Using user: andrei@zenml.io
Using stack: local-sagemaker-step-operator-wandb
  step_operator: sagemaker-eu
  container_registry: aws-eu
  experiment_tracker: zenml_wandb
  orchestrator: default
  image_builder: local
  artifact_store: s3-zenfiles
Step data_loader has started.
Dataset with 28 records loaded!
Step data_loader has finished in 8.308s.
Caching disabled explicitly for deploy_endpoint.
Step deploy_endpoint has started.
Found credentials in shared credentials file: ~/.aws/credentials
Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.
Creating model with name: sagemaker-scikit-learn-2024-02-12-17-02-10-887
Creating endpoint-config with name breast-cancer-classifier-2024-02-12-18-02-08-073179
Creating endpoint with name breast-cancer-classifier-2024-02-12-18-02-08-073179
------!Step deploy_endpoint has finished in 3m30s.
Step inference_preprocessor has started.
Yo

PipelineRunResponse(id=UUID('0db5aae1-8461-4b21-8ed0-8e8e5ae241d7'), permission_denied=False, body=PipelineRunResponseBody(created=datetime.datetime(2024, 2, 12, 17, 1, 45), updated=datetime.datetime(2024, 2, 12, 17, 1, 45), user=UserResponse(id=UUID('c915043d-706c-4885-9263-783cae8dad20'), permission_denied=False, body=UserResponseBody(created=datetime.datetime(2023, 11, 16, 16, 5, 33), updated=datetime.datetime(2024, 2, 12, 16, 13, 15), active=True, activation_token=None, full_name='Andrei', email_opted_in=True, is_service_account=False), metadata=None, name='andrei@zenml.io'), status=<ExecutionStatus.INITIALIZING: 'initializing'>, stack=StackResponse(id=UUID('5712467b-5ab0-4549-8bc9-2ea7714c8ed8'), permission_denied=False, body=StackResponseBody(created=datetime.datetime(2024, 1, 13, 12, 42, 57), updated=datetime.datetime(2024, 2, 8, 9, 59, 9), user=UserResponse(id=UUID('c6fcdcc8-69e1-4ff5-9eb2-6a53aa81a08b'), permission_denied=False, body=UserResponseBody(created=datetime.datetime(